#Step 1. Install EleutherAI Evaluations Harness
*   Logging into WandB is optional.
*   Logging into Huggingface API is required to run GPQA. This is to prevent database leakage.
*   Uses Goodfire API! Experimental as of Jan 2025

In [3]:
# Remove Google Colab specific import and install commands
import os
import huggingface_hub

# Install dependencies using pip in a separate cell if needed
!pip install goodfire
!pip install -e git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm]

Obtaining lm_eval[vllm,wandb] from git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] (from lm_eval[vllm,wandb])
  Updating c:\users\minh1\local\lm-evaluation-harness\src\lm-eval clone
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached accelerate-1.2.1-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached numexpr-2.10

DEPRECATION: git+https://github.com/menhguin/lm-evaluation-harness.git#egg=lm_eval[wandb,vllm] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git fetch -q --tags
  Running command git reset --hard -q 95ce7c50956fc0c1713b1fa7441fdf592b3ae56c
ERROR: Could not find a version that satisfies the requirement torch>=1.8 (from lm-eval) (from versions: none)
ERROR: No matching distribution found for torch>=1.8


Automated login for Hugging Face Hub via Colab Secrets. If you don't have this, it'll prompt for manual login if you don't have one. If you completely remove this, you can't run GPQA or use Llama models via HF.

In [20]:
# Install all necessary dependencies
!pip install --upgrade pip setuptools wheel
!pip uninstall wandb -y  # Remove any existing wandb installation
!pip install wandb --no-deps  # Install wandb without dependencies
!pip install docker-pycreds pathtools promise protobuf psutil requests sentry-sdk setproctitle typing-extensions  # Install wandb dependencies manually

  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [12]:
from dotenv import load_dotenv
import os

# Load .env file from specific path
load_dotenv("C:/Users/minh1/local/lm-evaluation-harness/venv/.env")

hf_token = os.getenv('HF_TOKEN')

if hf_token:
    huggingface_hub.login(hf_token)
else:
    print("Huggingface token not found. Please login manually.")
    !huggingface-cli login

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Automated login for WandB via Colab Secrets. If you don't have this, it'll just prompt you later if you use wandb_args.

In [19]:
# First install setuptools which includes distutils
!pip install setuptools
!pip install wandb

# Now try the WandB login
wandb_token = os.getenv('WANDB_API_KEY')

if wandb_token:
    os.environ["WANDB_API_KEY"] = wandb_token
    import wandb
    wandb.login()
else:
    print("WandB token not found. Continuing without logging into WandB.")

ModuleNotFoundError: No module named 'distutils'

Automated login for Goodfire API via Colab Secrets.

In [4]:
# Try to get GOODFIRE_API_KEY from environment or Colab secrets
GOODFIRE_API_KEY = os.getenv('GOODFIRE_API_KEY') or userdata.get('GOODFIRE_API_KEY')
if not GOODFIRE_API_KEY:
    raise ValueError("Please set GOODFIRE_API_KEY in environment or Colab secrets")

#Step 2. Run evaluation

In [9]:
# Cell 5
import os

# Set environment variable for Goodfire API key if not already set
if 'GOODFIRE_API_KEY' not in os.environ:
    os.environ['GOODFIRE_API_KEY'] = GOODFIRE_API_KEY  # Make sure this is defined from previous cells

# Run the evaluation using the command-line interface
!python -m lm_eval \
    --model goodfire \
    --model_args pretrained=meta-llama/Meta-Llama-3.1-8B-Instruct \
    --tasks gsm8k_cot_llama \
    --num_fewshot 8 \
    --limit 10 \
    --batch_size auto \
    --log_samples \
    --output_path ./lm-eval-output/ \
    --gen_kwargs top_p=0.9,temperature=1.0,do_sample=True

2025-01-11 15:46:36.174423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 15:46:36.198326: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 15:46:36.205320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 15:46:36.221929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-11 15:46:39.072227: W tensorflow/compiler/tf2

#Reference: EleutherAI Eval Harness task list
For those curious to run other evals! Please note that Min P is currently only accessible for `generate_until` tasks. There is currently no easy way to index these tasks, I just Ctrl + F'd `generate_until` on the [EleutherAI Evals Harness Repo](https://github.com/EleutherAI/lm-evaluation-harness).

In [6]:
# Test Goodfire Client
import goodfire

client = goodfire.Client(api_key=GOODFIRE_API_KEY)

# Simple test call
response = client.chat.completions.create(
    messages=[{"role": "user", "content": "Say hello!"}],
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_completion_tokens=10
)

# Access response using ChatCompletion object attributes
print("Test response:", response.choices[0].message['content'])

Test response: Hello! It's nice to meet you. How


In [ ]:
 !lm-eval --tasks list